In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
import networkx as nx

import numpy as np
import matplotlib.pyplot as plt

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
    
print('Using PyTorch version:', torch.__version__, ' Device:', device)

Using PyTorch version: 1.9.1  Device: cpu


In [2]:
batch_size = 32

train_dataset = datasets.MNIST('./data', 
                               train=True, 
                               download=True, 
                               transform=transforms.ToTensor())

validation_dataset = datasets.MNIST('./data', 
                                    train=False, 
                                    transform=transforms.ToTensor())

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

validation_loader = torch.utils.data.DataLoader(dataset=validation_dataset, 
                                                batch_size=batch_size, 
                                                shuffle=False)

/opt/anaconda3/lib/python3.8/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [3]:
for (X_train, y_train) in train_loader:
    print('X_train:', X_train.size(), 'type:', X_train.type())
    print('y_train:', y_train.size(), 'type:', y_train.type())
    break

X_train: torch.Size([32, 1, 28, 28]) type: torch.FloatTensor
y_train: torch.Size([32]) type: torch.LongTensor


In [4]:
class Net(nn.Module):
    def __init__(self, Net):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(28*28, 50)
        self.fc1_drop = nn.Dropout(0.2)
        self.fc2 = nn.Linear(50, 50)
        self.fc2_drop = nn.Dropout(0.2)
        self.fc3 = nn.Linear(50, 50)
        self.fc3_drop = nn.Dropout(0.2)
        self.fc4 = nn.Linear(50, 10)

# return list of all output of layers, name different variables for each layer (search for linear)
# only linear layer MLP for function, could add "weight mode" to change how weights are calculated
# will change for convolutional layers
    def forward(self, x):
        x = x.view(-1, 28*28)
        x = F.relu(self.fc1(x))
        x = self.fc1_drop(x)
        x = F.relu(self.fc2(x))
        x = self.fc2_drop(x)
        x = F.relu(self.fc3(x))
        x = self.fc3_drop(x)
        return F.log_softmax(self.fc4(x), dim=1)

model = Net(Net).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.5)
criterion = nn.CrossEntropyLoss()



print(model)

print(model.fc1.weight.size())
print(model.fc2.weight.size())
print(model.fc3.weight.size())
print(model.fc4.weight.size())

Net(
  (fc1): Linear(in_features=784, out_features=50, bias=True)
  (fc1_drop): Dropout(p=0.2, inplace=False)
  (fc2): Linear(in_features=50, out_features=50, bias=True)
  (fc2_drop): Dropout(p=0.2, inplace=False)
  (fc3): Linear(in_features=50, out_features=50, bias=True)
  (fc3_drop): Dropout(p=0.2, inplace=False)
  (fc4): Linear(in_features=50, out_features=10, bias=True)
)
torch.Size([50, 784])
torch.Size([50, 50])
torch.Size([50, 50])
torch.Size([10, 50])


In [5]:
def train(epoch, log_interval=200):
    # Set model to training mode
    model.train()
    
    # Loop over each batch from the training set
    for batch_idx, (data, target) in enumerate(train_loader):
        # Copy data to GPU if needed
        data = data.to(device)
        target = target.to(device)

        # Zero gradient buffers
        optimizer.zero_grad() 
        
        # Pass data through the network
        output = model(data)

        # Calculate loss
        loss = criterion(output, target)

        # Backpropagate
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data.item()))

In [6]:
def validate(loss_vector, accuracy_vector):
    model.eval()
    val_loss, correct = 0, 0
    for data, target in validation_loader:
        data = data.to(device)
        target = target.to(device)
        output = model(data)
        val_loss += criterion(output, target).data.item()
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    val_loss /= len(validation_loader)
    loss_vector.append(val_loss)

    accuracy = 100. * correct.to(torch.float32) / len(validation_loader.dataset)
    accuracy_vector.append(accuracy)
    
    print('\nValidation set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        val_loss, correct, len(validation_loader.dataset), accuracy))

In [7]:
%%time
epochs = 10

lossv, accv = [], []
for epoch in range(1, epochs + 1):
    train(epoch)
    validate(lossv, accv)

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.309115
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2.291416
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.210268
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1.810902
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1.380998
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.784937
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.838683
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.820605
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.536563
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.420858

Validation set: Average loss: 0.4362, Accuracy: 8752/10000 (88%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.490020
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.475165
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.295235
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.382350
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.266392
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.404854
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.360696
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.790635
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.60

In [8]:
#function for making graph from model
#layers = layers of model as list
#In the forward function, only out equals linear 1 and 2
#Two different variables, out1 = linear1 of 
def graph_from_model(layers):
    G = nx.DiGraph()
    #Create nodes of graph
    for layer_index in range(0,len(layers)):
        layer = layers[layer_index]
        for neuron_index in range(0,layer.weight.size()[1]):
            print("n"+str(layer_index)+"_"+str(neuron_index))
            G.add_node("n"+str(layer_index)+"_"+str(neuron_index))

        print("Layer "+str(layer_index)+" done")

    for neuron_index in range(0,layers[len(layers)-1].weight.size()[0]):
            print("n"+str(len(layers))+"_"+str(neuron_index))
            G.add_node("n"+str(len(layers))+"_"+str(neuron_index))
    print("Output layer done")

    #Create edges of graph
    for layer_index in range(0,len(layers)):
        layer = layers[layer_index]
        for x in range(0,layer.weight.size()[0]):
            for y in range(0,layer.weight.size()[1]):
                inNode = "n"+str(layer_index)+"_"+str(y)
                outNode = "n"+str(layer_index+1)+"_"+str(x)
                G.add_weighted_edges_from([(inNode, outNode, layer.weight[x][y].item())])
                print("Added edge from "+inNode+" to "+outNode+" with weight "+str(layer.weight[x][y].item()))
        print("Output layer edges done")

    return G

In [9]:
G = graph_from_model([model.fc1, model.fc2, model.fc3, model.fc4])

n0_0
n0_1
n0_2
n0_3
n0_4
n0_5
n0_6
n0_7
n0_8
n0_9
n0_10
n0_11
n0_12
n0_13
n0_14
n0_15
n0_16
n0_17
n0_18
n0_19
n0_20
n0_21
n0_22
n0_23
n0_24
n0_25
n0_26
n0_27
n0_28
n0_29
n0_30
n0_31
n0_32
n0_33
n0_34
n0_35
n0_36
n0_37
n0_38
n0_39
n0_40
n0_41
n0_42
n0_43
n0_44
n0_45
n0_46
n0_47
n0_48
n0_49
n0_50
n0_51
n0_52
n0_53
n0_54
n0_55
n0_56
n0_57
n0_58
n0_59
n0_60
n0_61
n0_62
n0_63
n0_64
n0_65
n0_66
n0_67
n0_68
n0_69
n0_70
n0_71
n0_72
n0_73
n0_74
n0_75
n0_76
n0_77
n0_78
n0_79
n0_80
n0_81
n0_82
n0_83
n0_84
n0_85
n0_86
n0_87
n0_88
n0_89
n0_90
n0_91
n0_92
n0_93
n0_94
n0_95
n0_96
n0_97
n0_98
n0_99
n0_100
n0_101
n0_102
n0_103
n0_104
n0_105
n0_106
n0_107
n0_108
n0_109
n0_110
n0_111
n0_112
n0_113
n0_114
n0_115
n0_116
n0_117
n0_118
n0_119
n0_120
n0_121
n0_122
n0_123
n0_124
n0_125
n0_126
n0_127
n0_128
n0_129
n0_130
n0_131
n0_132
n0_133
n0_134
n0_135
n0_136
n0_137
n0_138
n0_139
n0_140
n0_141
n0_142
n0_143
n0_144
n0_145
n0_146
n0_147
n0_148
n0_149
n0_150
n0_151
n0_152
n0_153
n0_154
n0_155
n0_156
n0_157
n0_1

Added edge from n0_23 to n1_1 with weight 0.023182500153779984
Added edge from n0_24 to n1_1 with weight 0.03324609622359276
Added edge from n0_25 to n1_1 with weight -0.028753209859132767
Added edge from n0_26 to n1_1 with weight 0.010730866342782974
Added edge from n0_27 to n1_1 with weight -0.005463877692818642
Added edge from n0_28 to n1_1 with weight 0.005537249147891998
Added edge from n0_29 to n1_1 with weight 0.03063507005572319
Added edge from n0_30 to n1_1 with weight 0.03291824832558632
Added edge from n0_31 to n1_1 with weight -0.011507712304592133
Added edge from n0_32 to n1_1 with weight 0.016869328916072845
Added edge from n0_33 to n1_1 with weight -0.025296492502093315
Added edge from n0_34 to n1_1 with weight -0.017827795818448067
Added edge from n0_35 to n1_1 with weight 0.024501174688339233
Added edge from n0_36 to n1_1 with weight 0.010742586106061935
Added edge from n0_37 to n1_1 with weight 0.016627809032797813
Added edge from n0_38 to n1_1 with weight 0.035356141

Added edge from n0_739 to n1_2 with weight -0.03240865468978882
Added edge from n0_740 to n1_2 with weight -0.05103503540158272
Added edge from n0_741 to n1_2 with weight -0.0115988003090024
Added edge from n0_742 to n1_2 with weight -0.039584681391716
Added edge from n0_743 to n1_2 with weight -0.046771444380283356
Added edge from n0_744 to n1_2 with weight 0.010241445153951645
Added edge from n0_745 to n1_2 with weight -0.04778606444597244
Added edge from n0_746 to n1_2 with weight -0.02728387527167797
Added edge from n0_747 to n1_2 with weight -0.036505915224552155
Added edge from n0_748 to n1_2 with weight 0.027239948511123657
Added edge from n0_749 to n1_2 with weight 0.02685759961605072
Added edge from n0_750 to n1_2 with weight -0.018022123724222183
Added edge from n0_751 to n1_2 with weight 0.0029235391411930323
Added edge from n0_752 to n1_2 with weight -0.019777698442339897
Added edge from n0_753 to n1_2 with weight -0.01559840515255928
Added edge from n0_754 to n1_2 with wei

Added edge from n0_670 to n1_4 with weight -0.029451869428157806
Added edge from n0_671 to n1_4 with weight -0.03543021157383919
Added edge from n0_672 to n1_4 with weight 0.014187335968017578
Added edge from n0_673 to n1_4 with weight 0.028534773737192154
Added edge from n0_674 to n1_4 with weight 0.035392045974731445
Added edge from n0_675 to n1_4 with weight -0.005770465359091759
Added edge from n0_676 to n1_4 with weight 0.007279986049979925
Added edge from n0_677 to n1_4 with weight -0.06930960714817047
Added edge from n0_678 to n1_4 with weight -0.08938179910182953
Added edge from n0_679 to n1_4 with weight -0.02961723878979683
Added edge from n0_680 to n1_4 with weight -0.005622083321213722
Added edge from n0_681 to n1_4 with weight -0.008402527309954166
Added edge from n0_682 to n1_4 with weight -0.013272075913846493
Added edge from n0_683 to n1_4 with weight 0.004989204462617636
Added edge from n0_684 to n1_4 with weight 0.043414000421762466
Added edge from n0_685 to n1_4 with

Added edge from n0_682 to n1_6 with weight -0.009705076925456524
Added edge from n0_683 to n1_6 with weight 0.0016922384966164827
Added edge from n0_684 to n1_6 with weight 0.047164782881736755
Added edge from n0_685 to n1_6 with weight 0.012037706561386585
Added edge from n0_686 to n1_6 with weight 0.03200157731771469
Added edge from n0_687 to n1_6 with weight 0.011930353008210659
Added edge from n0_688 to n1_6 with weight 0.024317918345332146
Added edge from n0_689 to n1_6 with weight 0.023270258679986
Added edge from n0_690 to n1_6 with weight 0.033085912466049194
Added edge from n0_691 to n1_6 with weight 0.002805875614285469
Added edge from n0_692 to n1_6 with weight 0.014471124857664108
Added edge from n0_693 to n1_6 with weight 0.023974092677235603
Added edge from n0_694 to n1_6 with weight 0.0352901965379715
Added edge from n0_695 to n1_6 with weight 0.01689615473151207
Added edge from n0_696 to n1_6 with weight 0.025441763922572136
Added edge from n0_697 to n1_6 with weight 0.

...

Added edge from n0_445 to n1_48 with weight -0.020035281777381897
Added edge from n0_446 to n1_48 with weight -0.03380408510565758
Added edge from n0_447 to n1_48 with weight -0.030813952907919884
Added edge from n0_448 to n1_48 with weight -0.016481606289744377
Added edge from n0_449 to n1_48 with weight 0.02239987626671791
Added edge from n0_450 to n1_48 with weight -0.03178582713007927
Added edge from n0_451 to n1_48 with weight -0.011557088233530521
Added edge from n0_452 to n1_48 with weight 0.006389325484633446
Added edge from n0_453 to n1_48 with weight -0.03810972347855568
Added edge from n0_454 to n1_48 with weight -0.07821192592382431
Added edge from n0_455 to n1_48 with weight 0.011281970888376236
Added edge from n0_456 to n1_48 with weight 0.014018416404724121
Added edge from n0_457 to n1_48 with weight -0.031976889818906784
Added edge from n0_458 to n1_48 with weight 0.02575693652033806
Added edge from n0_459 to n1_48 with weight -0.02126459591090679
Added edge from n0_460

Added edge from n1_24 to n2_9 with weight 0.14897756278514862
Added edge from n1_25 to n2_9 with weight -0.02236439846456051
Added edge from n1_26 to n2_9 with weight 0.016581296920776367
Added edge from n1_27 to n2_9 with weight -0.28614526987075806
Added edge from n1_28 to n2_9 with weight 0.10909763723611832
Added edge from n1_29 to n2_9 with weight -0.1937848925590515
Added edge from n1_30 to n2_9 with weight 0.022762171924114227
Added edge from n1_31 to n2_9 with weight 0.0203702375292778
Added edge from n1_32 to n2_9 with weight 0.17897169291973114
Added edge from n1_33 to n2_9 with weight -0.08373114466667175
Added edge from n1_34 to n2_9 with weight -0.07691983878612518
Added edge from n1_35 to n2_9 with weight -0.06552328914403915
Added edge from n1_36 to n2_9 with weight 0.19144730269908905
Added edge from n1_37 to n2_9 with weight -0.10295040160417557
Added edge from n1_38 to n2_9 with weight -0.13450433313846588
Added edge from n1_39 to n2_9 with weight 0.06641996651887894


Added edge from n1_4 to n2_38 with weight 0.026568174362182617
Added edge from n1_5 to n2_38 with weight -0.08896186947822571
Added edge from n1_6 to n2_38 with weight -0.09291087836027145
Added edge from n1_7 to n2_38 with weight -0.21774552762508392
Added edge from n1_8 to n2_38 with weight 0.03537874296307564
Added edge from n1_9 to n2_38 with weight 0.06995567679405212
Added edge from n1_10 to n2_38 with weight -0.06092189997434616
Added edge from n1_11 to n2_38 with weight -0.1774074286222458
Added edge from n1_12 to n2_38 with weight -0.1045851781964302
Added edge from n1_13 to n2_38 with weight -0.0073826368898153305
Added edge from n1_14 to n2_38 with weight 0.27022600173950195
Added edge from n1_15 to n2_38 with weight 0.07563460618257523
Added edge from n1_16 to n2_38 with weight 0.05661320686340332
Added edge from n1_17 to n2_38 with weight -0.17404714226722717
Added edge from n1_18 to n2_38 with weight -0.027906879782676697
Added edge from n1_19 to n2_38 with weight 0.24087

Added edge from n2_47 to n3_21 with weight -0.05055404081940651
Added edge from n2_48 to n3_21 with weight -0.08372229337692261
Added edge from n2_49 to n3_21 with weight 0.12509971857070923
Added edge from n2_0 to n3_22 with weight -0.19165413081645966
Added edge from n2_1 to n3_22 with weight 0.10724189877510071
Added edge from n2_2 to n3_22 with weight -0.08597765117883682
Added edge from n2_3 to n3_22 with weight -0.10565204918384552
Added edge from n2_4 to n3_22 with weight -0.21613554656505585
Added edge from n2_5 to n3_22 with weight -0.12574633955955505
Added edge from n2_6 to n3_22 with weight -0.17348013818264008
Added edge from n2_7 to n3_22 with weight 0.1707397997379303
Added edge from n2_8 to n3_22 with weight -0.14611928164958954
Added edge from n2_9 to n3_22 with weight -0.06792479753494263
Added edge from n2_10 to n3_22 with weight 0.2935728132724762
Added edge from n2_11 to n3_22 with weight -0.09983240067958832
Added edge from n2_12 to n3_22 with weight -0.0136487567

Added edge from n3_40 to n4_1 with weight -0.43901950120925903
Added edge from n3_41 to n4_1 with weight -0.09935477375984192
Added edge from n3_42 to n4_1 with weight 0.05646592006087303
Added edge from n3_43 to n4_1 with weight 0.3769841492176056
Added edge from n3_44 to n4_1 with weight -0.14887689054012299
Added edge from n3_45 to n4_1 with weight -0.09836293011903763
Added edge from n3_46 to n4_1 with weight -0.42439091205596924
Added edge from n3_47 to n4_1 with weight 0.5072692632675171
Added edge from n3_48 to n4_1 with weight 0.3113408088684082
Added edge from n3_49 to n4_1 with weight 0.1970681995153427
Added edge from n3_0 to n4_2 with weight -0.291143536567688
Added edge from n3_1 to n4_2 with weight -0.0391877144575119
Added edge from n3_2 to n4_2 with weight 0.14099209010601044
Added edge from n3_3 to n4_2 with weight 0.020476775243878365
Added edge from n3_4 to n4_2 with weight -0.16092504560947418
Added edge from n3_5 to n4_2 with weight -0.2528781294822693
Added edge f

In [11]:
betweenness = nx.betweenness_centrality(G, normalized=False, weight='weight')
vals = [v for _, v in betweenness.items()]
print(len(betweenness))

944


In [12]:
#Betweenness after training
betweenness_trained = nx.betweenness_centrality(G, normalized=False, weight='weight')
vals_trained = [v for _, v in betweenness_trained.items()]
print(sum(vals_trained))
print(len(vals_trained))
average_trained = sum(vals_trained) / len(vals_trained)
print(average_trained)
#Edge betweenness after training
betweenness_trained_edge = nx.edge_betweenness_centrality(G, normalized=False, weight='weight')
vals_trained_edge = [v for _, v in betweenness_trained_edge.items()]
print(sum(vals_trained_edge))
print(len(vals_trained_edge))
average_trained_edge = sum(vals_trained_edge) / len(vals_trained_edge)
print(average_trained_edge)

145120.0
944
153.72881355932202
279560.0
44700
6.25413870246085
